<a href="https://colab.research.google.com/github/aragaoiberis/Topic_modeling/blob/main/BERTopic_%2B_lbnlpR_TI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keyphrase-vectorizers
!pip install flair
!pip install bertopic
!pip install swifter 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import glob
import pandas as pd
import numpy as np
import swifter

import flair
from typing import List
from bertopic import BERTopic
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
from flair.embeddings import TransformerDocumentEmbeddings
from keyphrase_vectorizers import KeyphraseCountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = "/content/gdrive/MyDrive/lbnlpTI_R.csv"
df = pd.read_csv(path)

In [ ]:
df

,Unnamed: 0,PT,AU,AF,TI,AB,tags,TI_AB,TagsTI,SO,...,GP,SE,SP,BN,BE,SI,PN,HO,MA,SU
0,0,J,"['Bie, J', 'Yang, DB', 'Ju, MG', 'Pan, Q', 'Yo...","['Bie, Jie', 'Yang, Dai-Bei', 'Ju, Ming-Gang',...",Molecular Design of Three-Dimensional Metal-Fr...,The intense research activities on the hybrid ...,A(NH4)X-3 perovskites photovoltaic applications,Molecular Design of Three-Dimensional Metal-Fr...,"[[('molecular', 'O'), ('design', 'O'), ('of', ...",JACS AU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,J,"['Lu, SH', 'Zhou, QH', 'Ma, L', 'Guo, Y', 'Wan...","['Lu, Shuaihua', 'Zhou, Qionghua', 'Ma, Liang'...",Rapid Discovery of Ferroelectric Photovoltaic ...,Rapid discovery of novel functional materials ...,ferroelectric photovoltaic perovskites descrip...,Rapid Discovery of Ferroelectric Photovoltaic ...,"[[('rapid', 'O'), ('discovery', 'O'), ('of', '...",SMALL METHODS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,J,"['Attique, S', 'Ali, N', 'Rauf, S', 'Ali, S', ...","['Attique, Sanam', 'Ali, Nasir', 'Rauf, Sajid'...",Nontoxic and Less Toxic Hybrid Perovskites: a ...,"A research hotspot for the last decade, organo...",nontoxic perovskites photovoltaic devices,Nontoxic and Less Toxic Hybrid Perovskites: a ...,"[[('nontoxic', 'B-MAT'), ('and', 'O'), ('less'...",SOLAR RRL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,J,"['Aristizabal-Ferreira, VA', 'Guevara-Vela, JM...","['Alexander Aristizabal-Ferreira, Victor', 'Ma...",Computation of photovoltaic and stability prop...,Hybrid organic-inorganic perovskites (HOIPs) h...,photovoltaic stability properties perovskites,Computation of photovoltaic and stability prop...,"[[('computation', 'O'), ('of', 'O'), ('photovo...",THEORETICAL CHEMISTRY ACCOUNTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,J,"['Liu, G', 'Kong, LP', 'Yang, WG', 'Mao, HK']","['Liu, Gang', 'Kong, Lingping', 'Yang, Wenge',...",Pressure engineering of photovoltaic perovskites,As the driving force for technological innovat...,photovoltaic perovskites,Pressure engineering of photovoltaic perovskit...,"[[('pressure', 'O'), ('engineering', 'O'), ('o...",MATERIALS TODAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,5101,J,"['Yamanaka, S', 'Hayakawa, K', 'Cojocaru, L', ...","['Yamanaka, Soichiro', 'Hayakawa, Kei', 'Cojoc...",Electronic structures and chemical states of m...,Methylammonium lead triiodide (CH3NH3PbI3) is ...,electronic structures chemical states methylam...,Electronic structures and chemical states of m...,"[[('electronic', 'B-PRO'), ('structures', 'I-P...",JOURNAL OF APPLIED PHYSICS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5102,5102,J,"['Stoltzfus, DM', 'Donaghey, JE', 'Armin, A', ...","['Stoltzfus, Dani M.', 'Donaghey, Jenny E.', '...",Charge Generation Pathways in Organic Solar Ce...,Photocurrent generation in organic bulk hetero...,organic solar cells electron acceptor,Charge Generation Pathways in Organic Solar Ce...,"[[('charge', 'O'), ('generation', 'O'), ('path...",CHEMICAL REVIEWS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5103,5103,J,"['Wang, YB', 'Yue, YF', 'Yang, XD', 'Han, LY']","['Wang, Yanbo', 'Yue, Youfeng', 'Yang, Xudong'...",Toward Long-Term Stable and Highly Efficient P...,Perovskite solar cells (PSCs) have advanced qu...,perovskite solar cells charge transporting,Toward Long-Term Stable and Highly Efficient P...,"[[('toward', 'O'), ('long', 'O'), ('-', 'O'), ...",ADVANCED ENERGY MATERIALS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5104,5104,J,"['Yan, JF', 'Saunders, BR']","['Yan, Junfeng', 'Saunders, Brian R.']",Third-generation solar cells: a review and com...,The need for large scale low carbon solar elec...,third-generation solar cells perovskite solar ...,Third-generation solar cells: a review and com...,"[[('third', 'B-APL'), ('-', 'I-APL'), ('genera...",RSC ADVANCES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# define flair POS-tagger and splitter
tagger = SequenceTagger.load('pos-fast')
splitter = SegtokSentenceSplitter()


# define custom POS-tagger function using flair
def custom_pos_tagger(raw_documents: List[str], tagger: flair.models.SequenceTagger = tagger, splitter: flair.tokenization.SegtokSentenceSplitter = splitter)->List[tuple]:
    """
    Important: 

    The mandatory 'raw_documents' parameter can NOT be named differently and has to expect a list of strings. 
    Any other parameter of the custom POS-tagger function can be arbitrarily defined, depending on the respective use case. 
    Furthermore the function has to return a list of (word token, POS-tag) tuples.
    """ 
    # split texts into sentences
    sentences = []
    for doc in raw_documents:
        sent = splitter.split(doc)
        tagger.predict(sent)
        sentences.extend(sent)

    # predict POS tags

    # iterate through sentences to get word tokens and predicted POS-tags
    pos_tags = []
    words = []
    for sentence in sentences:
        pos_tags.extend([label.value for label in sentence.get_labels('pos')])
        words.extend([word.text for word in sentence])
    
    return list(zip(words, pos_tags))

2022-12-12 23:19:31,756 loading file /root/.flair/models/pos-english-fast/36f7923039eed4c66e4275927daaff6cd275997d61d238355fb1fe0338fe10a1.ff87e5b4e47fdb42a0c00237d9506c671db773e0a7932179ace82e584383a1b8
2022-12-12 23:19:32,348 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [ ]:
model = TransformerDocumentEmbeddings('sentence-transformers/allenai-specter')
vectorizer = KeyphraseCountVectorizer(custom_pos_tagger=custom_pos_tagger,pos_pattern = '<J.*>+<N.*>+')
topic_model = BERTopic(embedding_model=model,vectorizer_model=vectorizer,nr_topics = 10)

In [ ]:
df.tags.fillna("",inplace=True)

In [ ]:
keyphrase_topics, keyphrase_probs = topic_model.fit_transform(df.tags)

/usr/local/lib/python3.8/dist-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log((avg_nr_samples / df)+1)


In [ ]:
!pip install topically[bertopic]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from topically import Topically

In [ ]:
app = Topically('PE4bvbL37kORpnVCnTaZBLtfVgbVX4xunKpamWU0')

In [ ]:
#topics, probs = topic_model.fit_transform(df.TI)


In [ ]:
labels, mapper = app.name_topics((df.tags,keyphrase_topics))

In [ ]:
topic_model.set_topic_labels(mapper)

In [ ]:
topic_model.visualize_barchart(n_words=6,custom_labels=True,width=750,height=500)

In [ ]:
#hierarchical_topics = topic_model.hierarchical_topics(df.tags[:50],keyphrase_topics)

In [ ]:
topic_model.visualize_documents(df.tags,custom_labels=False,hide_annotations=False,width=2500,height=3000)

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
df.PY.fillna(2022,inplace=True)

In [ ]:
df.PY

0       2021.0
1       2019.0
2       2021.0
3       2022.0
4       2019.0
         ...  
5101    2018.0
5102    2016.0
5103    2018.0
5104    2014.0
5105    2018.0
Name: PY, Length: 5106, dtype: float64

In [ ]:
topics_over_time = topic_model.topics_over_time(df.TI,df.PY)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time,custom_labels=True)

In [ ]:
keyphrase_probs